In [1]:
import numpy as np
import matplotlib.pyplot as plt
import resfuncRead as rfr

from Yield_extract import Yield_NR, Yield_ER

In [2]:
#read the files (from anthony)
#ptres = rfr.getRFunc('data/jardin_ptres.txt')
#qres = rfr.getRFunc('data/jardin_qsummaxres.txt')

In [3]:
N = 10000
#Er = np.random.uniform(0,200,N)
#Er = np.random.exponential(40,np.uint32(N*0.3))
Er = 100 #for one Er 


ENR,N_e_h,sigQ,sigP,PT1,QNR,ER,Yield,U1,U2,fano = Yield_NR(N,Er)

Yield_er, ERer = Yield_ER(N)



ZeroDivisionError: float division by zero

In [ ]:
# Make band Fits (Thank you Anthony)

# files contain fits from previous data. I believe that they are fits from real data. As seen, the model has a significantly narrower spread the fits 
# from the data 

bpar_er = rfr.getBandFunc('data/erband_R133') #reads in band data/fit for er data
bpar_nr = rfr.getBandFunc('data/nrband_R133') #reads in band data/fit for nr data 

yer_mu = rfr.makeBFunc(bpar_er[1]['mu']) # sets average fit from 1st (2nd) col in data table. 
yer_muv = np.vectorize(yer_mu) #puts mean data for er in 1D array 
yer_sig = rfr.makeBFunc(bpar_er[1]['sig'],True) #sets uncertainty 
yer_sigv = np.vectorize(yer_sig) #puts uncertainty into 1D array

#following does the same but for nuclear recoils. 
ynr_mu = rfr.makeBFunc(bpar_nr[1]['mu'])
ynr_muv = np.vectorize(ynr_mu)
ynr_sig = rfr.makeBFunc(bpar_nr[1]['sig'],True)
ynr_sigv = np.vectorize(ynr_sig)

# try new fit

xn = np.arange(0.1,100,0.1)
m, c = np.polyfit(ER, Yield, 1)
yn = np.polyval([m, c], xn)

In [ ]:
#plot 


fig,axes = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)
ax1 = axes


X = np.arange(0.1,100,0.1)

ax1.plot(ER,Yield,'o',color='b',linewidth=2,markersize=1)
ax1.plot(Qer,Pter,'o',color='k',linewidth=2,markersize=1)
'''
ax1.plot(X,ynr_muv(X),color = 'orange',label='NR mu') #using old data 
#ax1.plot(xn,yn,'r--') #new fit to data (mean)

ax1.plot(X,ynr_muv(X)+3*ynr_sigv(X),'r-',label='NR 3$\sigma$')
ax1.plot(X,ynr_muv(X)-3*ynr_sigv(X),'r-',label=None)

ax1.plot(X,yer_muv(X),color='purple',linestyle='--',label='ER mu')
ax1.plot(X,yer_muv(X)+3*yer_sigv(X),color='purple',linestyle='-',label='ER 3$\sigma$')
ax1.plot(X,yer_muv(X)-3*yer_sigv(X),color='purple',linestyle='-',label=None)
'''
ax1.set_xlabel('Recoil Energy [keV]')
ax1.set_ylabel('Yield')
ax1.set_ylim(0,1.6)
#ax1.set_xlim(0,50)


plt.show()


In [ ]:
print(len(ER))
print(len(sigQ1))
print(len(sigP1))
print(len(Yield))
print(len(U1))
print(len(U2))
print(len(N_eh))


In [ ]:
#removing allowing Yield values from 0->1.6 and removing negative energies. Then creating energy bins. 

data = np.vstack((ER1,Yield,sigQ1,sigP1,U1,U2,N_eh)).T

x = data[data[:,1]>=0, :]

data1 = x[x[:,1]<=1.6,:]
data2 = data1[data1[:,0]>=0,:]
data1 = data2[np.argsort(data2[:, 0])]

#print(data2[:,0:3])
#print("-------------------------------------------------------")
#print(data1[:,0:3])

#a,b,c,d,e,f,g = np.array_split(data1,7)
data = np.array_split(data2,10)

#print(np.argsort(data2[:, 0]))

In [ ]:
xmin=min(Yield)
xmax=max(Yield)
amax= 30
#print(np.std(val_stat))
print(xmin,xmax)

n_res,n_resx = np.histogram(Yield,300,range=(xmin,xmax))


xresc = (n_resx[:-1] + n_resx[1:]) / 2

#set up a 1d plot
fig,axes = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)
ax1 = axes

X = np.arange(-amax,amax,0.01)

step, = ax1.step(xresc,n_res, where='mid',color='r', linestyle='-', label='Yield, Er = 40keV', linewidth=2)



ymin=0.1
ymax=max(n_res)


ax1.set_yscale('log')
ax1.set_xlim(xmin, xmax) #in pairs
ax1.set_ylim(ymin,ymax)
ax1.set_xlabel('A [a.u.]')
ax1.set_ylabel('counts')
ax1.grid(True)
ax1.yaxis.grid(True,which='minor',linestyle='--')
ax1.legend(loc=1,prop={'size':22})

for axis in ['top','bottom','left','right']:
  ax1.spines[axis].set_linewidth(2)

plt.tight_layout()
#plt.savefig('figures/figures')
plt.show()

In [ ]:
#Finds the width in a yield dist for a particular energy bin to be used later in yield variance calculation (fano)
def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def find_sigma(data):
    mu,sigma = norm.fit(data[:,1])
    n,bins = np.histogram(data[:,1],60)
    
    a = max(data[:,1])
    b = np.average(data[:,1])
    c = sigma
    
    popt,pcov = curve_fit(gaus,bins[:-1],n,p0=[a,b,c])
    
    return  pcov, popt

In [ ]:
'''
sig = []
for this_data in data:
    x,y = find_sigma(this_data)
    #print(x,y)
    sig.append(y[2])
    
print(sig)
'''

In [ ]:
#finds average fano in particular energy bin
def gaus1(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def find_fano(data):
    
    #mu,sigma = norm.fit(data)
    n,bins = np.histogram(data,60)
    
    a = max(data)
    b = np.mean(data)
    c = np.std(data)
    
    popt1,pcov1 = curve_fit(gaus1,bins[:-1],n,p0=[a,b,c])
    '''
    plt.figure()
    plt.hist(data,bins =60)
    plt.plot(bins,gaus1(bins,*popt1), 'r--', linewidth = 2)
    plt.xlabel("Extracted Fano")
    plt.show()
    '''

    
    
    #return  popt1, pcov1, c
    return b,c

In [ ]:
#Fano = (b[:,4]**2*b[:,2] +b[:,5]**2*b[:,3] -sigma)**2/(b[:,6]*(b[:,4]**2*eps + V/1000*b[:,5]**2))

def calc_fano(data):
    mean = []
    error = []
    fano = []
    stdy = []
    for x in data:
        #x = data[i]
        U1 = x[:,4]
        U2 = x[:,5]
        sigQ = x[:,2]
        sigP = x[:,3]
        N = x[:,6]
        sigy = np.std(x[:,1],0) #sample std 
        #Fano = (-U1**2*sigQ**2-U2**2*sigP**2 + sigy**2)/(U1**2*eps**2 +U2**2*(V/1000)**2)
        Fano = (sigy**2*Enr**2 - sigQ**2)/(eps**2*N)
        #Fano = Fano/N
        #print(min(x[:,0]),max(x[:,0]))
        
        y,z = find_fano(Fano)
        #print(Fano)
        
        n = len(x[:,1])
        h = np.mean(x[:,1]-np.mean(x[:,1]))
        
        err = np.sqrt((1/n)*((n-3)/(n-1))*sigy**4)*(2*sigy*Enr**2)/(eps**2*N)
        err1 = np.mean(err)
        error.append(err1)
        
        
        fano.append(Fano)
        mean.append(y)
        stdy.append(sigy)
        #error.append(np.std(Fano))
        
        #print(Fano)
       
    return mean, error,fano,stdy
        
        

In [ ]:
means,errors,fano,sigy= calc_fano(data)
#print(errors)

plt.figure()
plt.scatter(means,sigy)

x = np.arange(0,10)

plt.figure()
plt.errorbar(x,means,yerr=errors,fmt ='o',label = 'error', ecolor = 'purple', Linestyle = 'None', capsize=5, capthick=0.5)
plt.axhline(y=F, color='r', linestyle='-')
plt.ylim(0,150)
plt.show()